# Probabilistic Elastic FWI

In [1]:
import hmclab
import matplotlib.pyplot as plt
import numpy
import psvWave

from mpl_toolkits.axes_grid1 import AxesGrid

ModuleNotFoundError: No module named '__psvWave_cpp'

In [ ]:
fwi_settings = psvWave.fdModel.get_dictionary()

fwi_settings["domain"]["nt"] = 2400

fwi_settings["sources"]["delay_cycles_per_shot"] = 8

In [ ]:
psvWave.fdModel.write_dictionary("settings.ini", fwi_settings)

fdModel = psvWave.fdModel("settings.ini")

In [ ]:
for i_shot in range(fdModel.n_shots):
    fdModel.plot_domain(shot_to_plot=i_shot)
    plt.title(f"Shot {i_shot+1}")
    plt.show()

In [ ]:
# Create target model
# Get the coordinates of every grid point
IX, IZ = fdModel.get_coordinates(True)
# Get the associated parameter fields
vp, vs, rho = fdModel.get_parameter_fields()

x_middle = (IX.max() + IX.min()) / 2
z_middle = (IZ.max() + IZ.min()) / 2

# Add a circular negative anomaly to the 'true' model
circle = ((IX - x_middle) ** 2 + (IZ - z_middle) ** 2) ** 0.5 < 15
vs = vs * (1 - 0.1 * circle)
vp = vp * (1 - 0.1 * circle)
rho = rho * (1 + 0.1 * circle)

vp_target = vp
vs_target = vs
rho_target = rho

In [ ]:
background_means = numpy.array([2000, 800, 1500])
percentage_deviation = 0.2
mins = background_means * (1 - percentage_deviation)
maxs = background_means * (1 + percentage_deviation)

fdModel.set_parameter_fields(vp_target, vs_target, rho_target)
fdModel.plot_fields(vmin=mins, vmax=maxs)

In [ ]:
# Create 'true' data
# print("Faking observed data")
for i_shot in range(fdModel.n_shots):
    fdModel.forward_simulate(i_shot, omp_threads_override=6)

# Cheating of course, as this is synthetically generated data.
ux_obs, uz_obs = fdModel.get_synthetic_data()

numpy.random.seed(32312)
ux_obs += 25 * numpy.random.randn(*ux_obs.shape)
uz_obs += 25 * numpy.random.randn(*ux_obs.shape)

fdModel.plot_data(data=(ux_obs, uz_obs), exagerration=1)

In [ ]:
time_step = 80
plt.figure()
for i_shot in range(fdModel.n_shots):
    field = fdModel.get_snapshots()[0][i_shot, time_step, :, :]
    field_extr = numpy.max(numpy.abs(field))
    plt.imshow(field, vmin=-field_extr, vmax=field_extr, cmap=plt.get_cmap("seismic"))
    plt.colorbar()
    plt.title(f"Shot {i_shot+1}")
    plt.show()

In [ ]:
sampler = hmclab.Samplers.HMC()

posterior = hmclab.Distributions.ElasticFullWaveform2D(
    "settings.ini",
    ux_obs=ux_obs,
    uz_obs=uz_obs,
    omp_threads_override=6,
    temperature=25 * 25,
    free_parameter_grid=[10, 10],
)

posterior.update_bounds(
    lower_bounds=posterior.get_model_vector() - 200,
    upper_bounds=posterior.get_model_vector() + 200,
)


sampler.sample(
    "samples.h5",
    posterior,
    autotuning=True,
    initial_model=posterior.get_model_vector(),
    amount_of_steps=10,
    proposals=10000,
)

In [ ]:
sampler.stepsizes

In [ ]:
with hmclab.Samples("samples.h5") as samples:
    samples_n = samples.numpy

In [ ]:
fdModel.plot_model_vector(
    posterior.blob.forward_transform_background(
        samples_n[:-1, 100:].mean(axis=1)[:, None]
    )
)

fdModel.plot_model_vector(
    posterior.blob.forward_transform_background(
        samples_n[:-1, 100:].std(axis=1)[:, None]
    )
)

In [ ]:
_ = plt.plot(samples_n[:-1, :].T)

In [ ]:
m = samples_n[:-1, :].mean(axis=1)

In [ ]:
vector_splits = numpy.split(m, 3)

fields = fdModel.get_parameter_fields()
for i, field in enumerate(fields):
    field[:] = numpy.nan
    field[
        (fdModel.nx_inner_boundary + fdModel.np_boundary) : -(
            fdModel.nx_inner_boundary + fdModel.np_boundary
        ),
        (fdModel.nz_inner_boundary + fdModel.np_boundary) : -(
            fdModel.nz_inner_boundary + fdModel.np_boundary
        ),
    ] = (
        vector_splits[i]
        .reshape(
            (
                fdModel.nz_free_parameters,
                fdModel.nx_free_parameters,
            )
        )
        .T
    )

In [ ]:
from scipy.ndimage import gaussian_filter

plt.imshow(gaussian_filter(fields[1][35:-35, 35:-35], 3), cmap=plt.get_cmap("seismic"))
plt.colorbar()